In [1]:
from pyspark.sql.functions import col, sha2, concat_ws, current_timestamp,regexp_replace, avg,when, length

In [3]:
customer_schema='member_id string,emp_title string,emp_length string,home_ownership string,annual_inc float,addr_state string,zip_code string,country string,grade string,sub_grade  string,verification_status string,tot_hi_cred_lim float,application_type string,annual_inc_joint float,verification_status_joint string'

In [ ]:
srv_date = dt.datetime.now().strftime('%Y-%m-%d')
print(srv_date)
custPath = f"/user/{user_name}/lendingClub/srv/2024-01-07/customers/"
print(custPath)
csv_dest = f"/user/{user_name}/lendingClub/bv/csv/customers/"
parq_dest = f"/user/{user_name}/lendingClub/bv/parquet/customers"
print(csv_dest)
print(parq_dest)

In [ ]:
custDF = spark.read \
            .format("csv") \
            .option("inferSchema",True) \
            .option("header",True) \
            .option("path",custPath) \
            .load()

In [ ]:
rn_custDF = custDF.withColumnRenamed("annual_inc","annual_income") \
                    .withColumnRenamed("addr_state","address_state") \
                    .withColumnRenamed("zip_code","address_zipcode") \
                    .withColumnRenamed("country","address_country") \
                    .withColumnRenamed("tot_hi_cred_lim","total_high_credit_limit") \
                    .withColumnRenamed("annual_inc_joint","join_annual_income")

In [ ]:
id_DF = rn_custDF.withColumn("ingest_date",current_timestamp())


In [ ]:
un_custDF = id_DF.dropDuplicates()
fil_custDF = un_custDF.dropna(subset=["annual_income"])
emp_len_custDF = fil_custDF.withColumn("emp_length",regexp_replace(col("emp_length"),"(\D)","").cast("int"))


In [ ]:
avg_emp_len = emp_len_custDF.select(avg("emp_length").cast("int")).collect()
avg_emp_ln = avg_emp_len[0][0]


In [ ]:
avg_el_DF = emp_len_custDF.na.fill(avg_emp_ln,subset=["emp_length"])

In [ ]:
as_DF = avg_el_DF.withColumn("address_state", when (length(col("address_state"))> 2, "NA") \
                                        .otherwise(col("address_state"))
                    )

In [ ]:
as_DF.write \
        .format("csv") \
        .mode("overwrite") \
        .option("header",True) \
        .option("path", csv_dest) \
        .save()

csv_cust_count = as_DF.count()
addLog("ingested customers file with " + str(csv_cust_count) + " records in BV")

In [ ]:
as_DF.write \
        .format("parquet") \
        .option("header",True) \
        .option("path", parq_dest) \
        .mode("overwrite") \
        .save()

In [ ]:
cust_stat = "Success"